In [11]:
import numpy as np

In [2]:
def is_float(element: any) -> bool:
    if element is None: 
        return False
    try:
        float(element)
        return True
    except ValueError:
        return False
    

In [3]:
def read_from_string(string):
    data = string.split('\n')
    allowed_signs = {'<=','>='}
    if data[0] == 'min':
        min_max_multiplicator = -1
    elif data[0] == 'max':
        min_max_multiplicator = 1
        
    else:
        raise AssertionError('Введите направление оптимизации в первой строке файла')
    c = list(map(float, data[1].split()))
    A = []
    b = []
    for row in data[2:]:
        cur_row = []
        cur_sign = 0 
        for val in row.split():
#             try:
#                 val=float(val)
#             except:
#                 pass
#             print(val)
            if is_float(val):
                
                cur_row.append(float(val))
            elif val in allowed_signs and cur_sign == 0:
                if val == '<=':
                    cur_sign = 1
                else:
                    cur_sign = -1
            else:
                raise AssertionError(f'Неправильный формат строки {row}')
        if cur_sign != 0:
            cur_row = np.array(cur_row) * cur_sign
        else:
            raise AssertionError(f'Неправильный формат строки, знак неравенства не найден {row} \n Допустимые форматы знаков неравенств: {allowed_signs}')
        print(cur_row)
        A.append(cur_row[:-1])
        b.append(cur_row[-1])
    A = np.array(A)
    b = np.array(b).reshape(-1,1)
    c = np.array(c).reshape(1, -1) * min_max_multiplicator
    A=A.tolist()
    b=b[:,0].tolist()
    c=c.tolist()[0]
    return A, b, c



In [4]:
def to_tableau(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]

In [5]:
def can_be_improved(tableau):
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])

In [6]:
import math

def get_pivot_position(tableau):
    z = tableau[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    
    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column

In [7]:
def pivot_step(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]
    
    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier
   
    return new_tableau

In [8]:
def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
#         print('col',column)
        solutions.append(solution)
        
    return solutions

In [9]:
def simplex(c, A, b):
    tableau = to_tableau(c, A, b)   

    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)

In [72]:
A, b, c=read_from_string(
"""max
100 120
105 110 <= 10000
69 70 <= 9000
5 6 <= 10000
1 0 >= 0
0 1 >= 0""")
A, b, c

[  105.   110. 10000.]
[  69.   70. 9000.]
[5.e+00 6.e+00 1.e+04]
[-1. -0. -0.]
[-0. -1. -0.]


([[105.0, 110.0], [69.0, 70.0], [5.0, 6.0], [-1.0, -0.0], [-0.0, -1.0]],
 [10000.0, 9000.0, 10000.0, -0.0, -0.0],
 [100.0, 120.0])

In [73]:
solution = simplex(c, A, b)
print('solution: ', solution)

solution:  [0, 90.9090909090909, 0]
